In [1]:
import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
from io import BytesIO

In [14]:
# Параметры
repo_id = "ajtkulov/telegram-ru"
meta_zip_filename = "meta/all.channels.csv.zip"          # путь в репозитории
output_meta_dir = "data/posts/ajtkulov/meta"             # куда сохраняем

os.makedirs(output_meta_dir, exist_ok=True)

# Шаг 1: Скачивание ZIP-файла
try:
    zip_path = hf_hub_download(
        repo_id=repo_id,
        filename=meta_zip_filename,
        repo_type="dataset"
    )
    print(f"ZIP-файл скачан: {zip_path}")
except Exception as e:
    print(f"Ошибка скачивания: {e}")
    raise

# Шаг 2: Распаковка в нужную папку
print("Распаковка в папку:", output_meta_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    inner_files = zip_ref.namelist()
    print(f"Файлы внутри ZIP: {inner_files}")
    
    if not inner_files:
        raise ValueError("ZIP-файл пустой")
    
    zip_ref.extractall(output_meta_dir)
    print("Распаковка завершена")

# Шаг 3: Поиск распакованного CSV-файла
csv_path = None
for file in os.listdir(output_meta_dir):
    if file.lower().endswith('.csv'):
        csv_path = os.path.join(output_meta_dir, file)
        break

if not csv_path:
    raise FileNotFoundError("CSV-файл не найден после распаковки. Проверьте папку.")

print(f"\nЧтение CSV из файла: {csv_path}")

# Шаг 4: Чтение с обработкой всех типичных проблем
encodings = ['utf-8-sig', 'cp1251', 'utf-8', 'latin1', 'iso-8859-1']
df_channels = None

for encoding in encodings:
    try:
        df_channels = pd.read_csv(
            csv_path,
            encoding=encoding,
            sep='\t',                  # табуляция (как у вас)
            on_bad_lines='skip',       # пропуск битых строк
            low_memory=False,
            encoding_errors='replace'  # замена кракозябр
        )
        print(f"\nУСПЕХ! Файл прочитан с кодировкой: {encoding}")
        break
    except Exception as e:
        print(f"Попытка с {encoding} провалилась: {str(e)}")

if df_channels is None:
    raise ValueError("Не удалось прочитать CSV ни одной кодировкой. Возможно файл повреждён.")

# ← Добавляем названия колонок
column_names = ["link", "name", "description", "category", "message_id"]
if len(df_channels.columns) == len(column_names):
    df_channels.columns = column_names
    print("\nКолонки успешно переименованы в: link, name, description, category, message_id")
else:
    print(f"\nВнимание! Количество столбцов ({len(df_channels.columns)}) не равно ожидаемому ({len(column_names)}).")
    print("Колонки НЕ переименованы. Текущие:", df_channels.columns.tolist())


# Общая статистика
print(f"Количество каналов: {len(df_channels):,}")
print(f"Количество столбцов: {len(df_channels.columns)}")
print("Столбцы:", df_channels.columns.tolist())

# Первые 10 строк
print("\nПервые 10 каналов:")
display(df_channels.head(10))

# Распределение категорий
if 'category' in df_channels.columns:
    print("\nРаспределение по category (топ-15):")
    print(df_channels['category'].value_counts().head(150))
else:
    print("\nКолонка 'category' не найдена")

if 'category' in df_channels.columns:
    unique_categories = df_channels['category'].dropna().unique()
    unique_df = pd.DataFrame(unique_categories, columns=['category'])
    
    unique_csv_path = os.path.join(output_meta_dir, "unique_categories.csv")
    unique_df.to_csv(unique_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\nУникальные категории сохранены в файл: {unique_csv_path}")
    print(f"Количество уникальных категорий: {len(unique_categories)}")
    print("Первые 20 уникальных категорий:")
    print(unique_df.head(20))
else:
    print("\nНе удалось сохранить категории — колонка 'category' отсутствует")

print("CSV лежит здесь:", csv_path)

ZIP-файл скачан: C:\Users\Admin\.cache\huggingface\hub\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\meta\all.channels.csv.zip
Распаковка в папку: data/posts/ajtkulov/meta
Файлы внутри ZIP: ['all.channels.csv']
Распаковка завершена

Чтение CSV из файла: data/posts/ajtkulov/meta\all.channels.csv

УСПЕХ! Файл прочитан с кодировкой: utf-8-sig

Колонки успешно переименованы в: link, name, description, category, message_id
Количество каналов: 380,467
Количество столбцов: 5
Столбцы: ['link', 'name', 'description', 'category', 'message_id']

Первые 10 каналов:


,link,name,description,category,message_id
0,https://tgstat.ru/channel/@premium,Telegram Premium,Telegram Premium – a subscription that unlocks...,Telegram,7980100.0
1,https://tgstat.ru/channel/sOj9iDAtUkMyYWQy,Топор Live,"Нейтрально, без пропаганды. Топор Live с быстр...",Новости,4388359.0
2,https://tgstat.ru/channel/@wewantyoutodothejob,WeWantYou,Канал для поиска исполнителей для разных задач...,Другое,4158678.0
3,https://tgstat.ru/channel/@leoday,Леонардо Дайвинчик,Бот знакомств @leomatchbot,Юмор и развлечение,4057819.0
4,https://tgstat.ru/channel/@novosti_efir,Прямой Эфир • Новости,️Все самое важное в одном канале. Новости Росс...,Новости,3886208.0
5,https://tgstat.ru/channel/@novosti_voinaa,СМИ Россия не Москва,Эруктации информпространства России и ее Окраин.,Новости,3368394.0
6,https://tgstat.ru/channel/@rian_ru,РИА Новости,Главные Новости РИА t.me/rian_ru,Новости,3220976.0
7,https://tgstat.ru/channel/@invest_zonaa,INVEST ZONE,"Привет! Я Руслан, с 2017 года торгую рынок кри...",Криптовалюты,3057506.0
8,https://tgstat.ru/channel/@mash,Mash,"Прислать новость, фото, видео, аудио, бересту:...",Новости,2827677.0
9,https://tgstat.ru/channel/@crypto_drop_stukach,Дропы от Стукача,"Все о крипто раздачах, прибыльных темах и абуз...",Шок-конент,2636192.0



Распределение по category (топ-15):
category
Новости                                                               25909
Блоги                                                                 25599
Другое                                                                20693
Мода и красота                                                        17673
Психология                                                            13740
                                                                      ...  
Политика|||Регион|||Свердловская область                                 19
Политика|||Регион|||Самарская область                                    19
Путешествия|||Регион|||Приморский край                                   18
Новости|||Регион|||Кабардино-Балкарская Республика|||Новости и СМИ       18
Политика|||Регион|||Пермский край                                        18
Name: count, Length: 150, dtype: int64

Уникальные категории сохранены в файл: data/posts/ajtkulov/meta\unique_categor

In [ ]:


# Параметры (настройте под себя)
repo_id = "ajtkulov/telegram-ru"
output_dir = "data/posts/ajtkulov"           # Папка для результатов (CSV/JSONL)
cache_dir = os.path.join(output_dir, "cache")  # Кэш для ZIP (можно удалять после обработки)
progress_file = os.path.join(output_dir, "progress.json")  # Для паузы/продолжения

os.makedirs(output_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "csv"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "jsonl"), exist_ok=True)

# Критерии фильтра (пример; настройте)
min_text_length = 100
keywords = []  # Если пусто — без фильтра по словам; иначе содержит хотя бы одно
min_date = "2020-01-01"  # Формат YYYY-MM-DD

# Список всех файлов
file_names = [f"tg.{i}.zip" for i in range(359)]  # 0 to 112

# Для теста: file_names = file_names[:3]  # Только первые 3

# Загрузка прогресса (какие ZIP уже обработаны)
processed_files = set()
if os.path.exists(progress_file):
    with open(progress_file, 'r') as pf:
        processed_files = set(json.load(pf))
    print(f"Загружен прогресс: {len(processed_files)} файлов уже обработано")

# Основной цикл: последовательная обработка
for filename in tqdm(file_names, desc="Обработка ZIP-файлов"):
    if filename in processed_files:
        print(f"\n{filename} уже обработан — пропуск")
        continue
    
    print(f"\n=== Обработка {filename} ===")
    
    try:
        # Шаг 1: Скачивание (потоково, но hf_hub_download скачивает целиком)
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"data/{filename}",
            repo_type="dataset",
            cache_dir=cache_dir
        )
        print(f"Скачано в: {file_path}")
        
        # Шаг 2: Потоковая обработка ZIP
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            inner_files = [f for f in zip_ref.namelist() if not f.endswith('/') and not f.endswith('.done')]
            print(f"Файлов внутри: {len(inner_files)}")
            
            filtered_posts = []  # Временный буфер для этого ZIP (маленький, чтобы не жрать RAM)
            
            for inner_path in tqdm(inner_files, desc="Файлы внутри ZIP", leave=False):
                with zip_ref.open(inner_path) as f:
                    # Потоковое чтение строк
                    for line in TextIOWrapper(f, encoding='utf-8', errors='ignore'):
                        line = line.strip()
                        if not line: continue
                        
                        try:
                            obj = json.loads(line)
                            text = obj.get('text', '')
                            date = obj.get('date', '')
                            
                            # Фильтр
                            if (len(text) >= min_text_length and
                                (not keywords or any(kw.lower() in text.lower() for kw in keywords)) and
                                date >= min_date):
                                filtered_posts.append(obj)
                        except json.JSONDecodeError:
                            continue  # Пропуск битых строк
            
            # Шаг 3: Сохранение отфильтрованных из этого ZIP
            if filtered_posts:
                df = pd.DataFrame(filtered_posts)
                batch_name = filename.replace('.zip', '')
                
                # CSV
                csv_path = os.path.join(output_dir, "csv", f"{batch_name}.csv")
                df.to_csv(csv_path, index=False, encoding='utf-8-sig')
                print(f"CSV: {csv_path} ({len(df):,} постов)")
                
                # JSONL
                jsonl_path = os.path.join(output_dir, "jsonl", f"{batch_name}.jsonl")
                with open(jsonl_path, 'w', encoding='utf-8') as jf:
                    for post in filtered_posts:
                        jf.write(json.dumps(post, ensure_ascii=False) + '\n')
                print(f"JSONL: {jsonl_path} ({len(filtered_posts):,} постов)")
            else:
                print("Нет подходящих постов в этом ZIP")
        
        # Опционально: удалить ZIP после обработки
        os.remove(file_path)
        print(f"ZIP удалён: {file_path}")
        
        # Сохранить прогресс
        processed_files.add(filename)
        with open(progress_file, 'w') as pf:
            json.dump(list(processed_files), pf)
        print("Прогресс сохранён")
        
    except Exception as e:
        print(f"Ошибка с {filename}: {e}")
        # Здесь можно добавить паузу или break, но код продолжит с следующим

print("\nОбработка завершена!")

Загружен прогресс: 2 файлов уже обработано


Обработка ZIP-файлов:   0%|          | 0/359 [00:00<?, ?it/s]


tg.0.zip уже обработан — пропуск

tg.1.zip уже обработан — пропуск

=== Обработка tg.2.zip ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tg.2.zip:  18%|#8        | 241M/1.31G [00:00<?, ?B/s]

KeyboardInterrupt: 